In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import datasets

from peft import PromptEncoderConfig, TaskType, get_peft_model

import opacus
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus import PrivacyEngine

import torch
import torch.nn as nn
import numpy as np

from tqdm.notebook import tqdm
from torch.optim import SGD
from torch.utils.data import DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, DataCollatorWithPadding

from sklearn.metrics import accuracy_score

In [ ]:
model_name = "prajjwal1/bert-tiny"
EPOCHS = 5
BATCH_SIZE = 16
LR = 2e-5

In [ ]:
dataset = datasets.load_dataset('glue', 'sst2')
num_labels = dataset["train"].features["label"].num_classes

In [ ]:
padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name)

if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    outputs = tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)
    return outputs

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence"],
)

tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], 
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    batch_size=BATCH_SIZE, 
    collate_fn=data_collator
)

In [ ]:
EPSILON = 8.0
DELTA = 1/len(train_dataloader)
MAX_GRAD_NORM = 0.01
MAX_PHYSICAL_BATCH_SIZE = int(BATCH_SIZE/4)

In [ ]:
config = AutoConfig.from_pretrained(model_name)
config.num_labels = num_labels

peft_config = PromptEncoderConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=32,
    encoder_hidden_size=128
)

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
model = get_peft_model(model, peft_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
errors = ModuleValidator.validate(model, strict=False)
print(errors)

In [ ]:
optimizer = SGD(params=model.parameters(), lr=LR)

In [ ]:
privacy_engine = PrivacyEngine(accountant="rdp")

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
    batch_first=True,
)

In [ ]:
print(f"Using Sigma = {optimizer.noise_multiplier:.3f} | C = {optimizer.max_grad_norm} | Initial DP (ε, δ) = ({privacy_engine.get_epsilon(DELTA)}, {DELTA})")

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable Parameters: {trainable_params} || All Parameters: {all_param} || Trainable Parameters (%): {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

In [ ]:
def train(model, train_dataloader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    epsilon = []

    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer,
        ) as memory_safe_data_loader:

        for i, batch in tqdm(enumerate(memory_safe_data_loader), total=len(memory_safe_data_loader), desc=f"Training Epoch: {epoch}"):
            
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()

            outputs = model(**batch)
            loss = criterion(outputs.logits, batch["labels"])
            loss.backward()

            optimizer.step()
            losses.append(loss.item())

            if i % 8000 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)

                print(f"Training Epoch: {epoch} | Loss: {np.mean(losses):.6f} | ε = {epsilon:.2f}")

In [ ]:
def test(model, test_dataloader, device):
    model.eval()
    eval_loss = 0
    eval_preds = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Test"):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
        loss = outputs.loss

        eval_loss += loss.detach().float()
        preds = outputs.logits.argmax(dim=-1)

        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )
        acc = accuracy_score(preds.cpu().numpy(), batch["labels"].cpu().numpy())

    print(
        f"Test set: Loss: {eval_loss:.4f}, Accuracy: {acc*100:.2f}%"
    )

    return eval_loss, acc

In [ ]:
for epoch in tqdm(range(EPOCHS), desc=f'Training {EPOCHS} Epochs'):
    train(model, train_dataloader, optimizer, epoch + 1, device)

In [ ]:
final_epsilon = privacy_engine.get_epsilon(DELTA)
print(f"Final DP Guarantee (ε, δ)-DP = ({final_epsilon:.2f}, {DELTA})")

In [ ]:
test(model, test_dataloader, device)